# MC-SVD Procedure - Node Classification

In [37]:
import torch
import torch.nn as nn
import numpy as np
from torch.nn import init
from random import shuffle, randint
import torch.nn.functional as F
from torch_geometric.datasets import Reddit, PPI, Planetoid
from itertools import combinations, combinations_with_replacement
from sklearn.metrics import f1_score, accuracy_score
from sklearn.decomposition import TruncatedSVD

## Define the dataset, the type of prediction and the number of samples

In [41]:
DATASET = 'cora'
PREDICTION = 'node'
PATH_TO_DATASETS_DIRECTORY = './'

In [201]:
# t-SVD sampling parameters
NUM_SAMPLES = 1
NUM_ITERS   = 1

In [202]:
datasets = {
    'reddit': Reddit(root=PATH_TO_DATASETS_DIRECTORY + '/datasets/Reddit'),
    'cora' : Planetoid(root=PATH_TO_DATASETS_DIRECTORY + '/datasets/Cora/', name='Cora'),
    'citeseer' : Planetoid(root=PATH_TO_DATASETS_DIRECTORY + '/datasets/CiteSeer/', name='CiteSeer'),
    'pubmed' : Planetoid(root=PATH_TO_DATASETS_DIRECTORY + '/datasets/PubMed/', name='PubMed'),
}
dataset = datasets[DATASET]
data = dataset[0]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [203]:
predictions = {
    'node' : dataset.num_classes,
    'link' : 2,
    'triad' : 4,
}

In [204]:
print("Printing Dataset Characteristics")
print("Name: ", DATASET)
print("Total Number of Nodes: ", data.num_nodes)
print("Total Number of Training Nodes: ", data.train_mask.sum().item())
print("Total Number of Val Nodes: ", data.val_mask.sum().item())
print("Total Number of Test Nodes: ", data.test_mask.sum().item())
print("Num Node Features: ", data.num_features)
print("Num Node Classes: ", dataset.num_classes)
print("Number of Edges: ", data.edge_index.shape[1])
print("Number of Samples for structural: ", NUM_SAMPLES)
print("Prediction Type: ", PREDICTION)

Printing Dataset Characteristics
Name:  cora
Total Number of Nodes:  2708
Total Number of Training Nodes:  140
Total Number of Val Nodes:  500
Total Number of Test Nodes:  1000
Num Node Features:  1433
Num Node Classes:  7
Number of Edges:  10556
Number of Samples for structural:  1
Prediction Type:  node


In [205]:
# data.train_mask = 1 - data.val_mask - data.test_mask
data.train_mask = ~data.val_mask * ~data.test_mask

adj_mat = torch.zeros((data.num_nodes,data.num_nodes))
edges = data.edge_index.t()
adj_mat[edges[:,0], edges[:,1]] = 1

## Build the non-overlapping induced subgraphs

In [206]:
adj_train = adj_mat[data.train_mask].t()[data.train_mask].t()
adj_validation = adj_mat[data.val_mask].t()[data.val_mask].t()
adj_test = adj_mat[data.test_mask].t()[data.test_mask].t()

## Corrupt a small fraction of the edges

In [207]:
def corrupt_adj(adj_mat, task, percent=2):
    """ Returns the corrupted version of the adjacency matrix """
    if task == 'link':
        edges = adj_mat.triu().nonzero()
        num_edges = edges.shape[0]
        num_to_corrupt = int(percent/100.0 * num_edges)
        random_corruption = np.random.randint(num_edges, size=num_to_corrupt)
        adj_mat_corrupted = adj_mat.clone()
        false_edges, false_non_edges = [], []
        #Edge Corruption
        for ed in edges[random_corruption]:
            adj_mat_corrupted[ed[0], ed[1]] = 0
            adj_mat_corrupted[ed[1], ed[0]] = 0
            false_non_edges.append(ed.tolist())
        #Non Edge Corruption
        random_non_edge_corruption = list(np.random.randint(adj_mat.shape[0], size = 6*num_to_corrupt))
        non_edge_to_corrupt = []
        for k in range(len(random_non_edge_corruption)-1):
            to_check = [random_non_edge_corruption[k], random_non_edge_corruption[k+1]]
            if to_check not in edges.tolist():
                non_edge_to_corrupt.append(to_check)
            if len(non_edge_to_corrupt) == num_to_corrupt:
                break
        non_edge_to_corrupt = torch.Tensor(non_edge_to_corrupt).type(torch.int16)
        for n_ed in non_edge_to_corrupt:
            adj_mat_corrupted[n_ed[0], n_ed[1]] = 1
            adj_mat_corrupted[n_ed[1], n_ed[0]] = 1
            false_edges.append(n_ed.tolist())
    return adj_mat_corrupted, false_edges, false_non_edges


In [208]:
adj_train_corrupted, train_false_edges, train_false_non_edges = corrupt_adj(adj_train, 'link', percent=2)
adj_val_corrupted, val_false_edges, val_false_non_edges = corrupt_adj(adj_validation, 'link', percent=2)
adj_test_corrupted, test_false_edges, test_false_non_edges  = corrupt_adj(adj_test, 'link', percent=2)

## Define the Supervised Learning Network

In [209]:
num_neurons = 256
input_rep = num_neurons + data.num_features

class StructMLP(nn.Module):
    """
        Compute an estimate of the expected value of a function of node embeddings
        Permutation Invariant Function - Deepsets - Zaheer, et al.
    """
    def __init__(self, node_set_size=1):
        super(StructMLP, self).__init__()

        self.node_set_size = node_set_size
        #Deepsets MLP

        self.ds_layer_1 = nn.Linear(input_rep*node_set_size, num_neurons)
        self.ds_layer_2 = nn.Linear(num_neurons, num_neurons)

        #One Hidden Layer
        self.layer1 = nn.Linear(num_neurons, num_neurons)
        self.layer2 = nn.Linear(num_neurons, predictions[PREDICTION])
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        #Deepsets initially on each of the samples
        num_nodes = input_tensor.shape[1]
        comb_tensor = torch.LongTensor(list(combinations(range(num_nodes), self.node_set_size)))
        sum_tensor = torch.zeros(comb_tensor.shape[0], num_neurons).to(device)

        for i in range(input_tensor.shape[0]):
            #Process the input tensor to form n choose k combinations and create a zero tensor
            set_init_rep = input_tensor[i][comb_tensor].view(comb_tensor.shape[0],-1)

            x = self.ds_layer_1(set_init_rep)
            x = self.relu(x)
            x = self.ds_layer_2(x)
            sum_tensor += x

        x = sum_tensor / input_tensor.shape[0]

        #One Hidden Layer for predictor
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

    def compute_loss(self, input_tensor, target):
        pred = self.forward(input_tensor)
        return F.cross_entropy(pred, target)

In [210]:
if PREDICTION == 'node':
    node_set_size = 1
elif PREDICTION == 'link':
    node_set_size = 2
else:
    node_set_size = 3

mlp = StructMLP(node_set_size).to(device)
mlp_optimizer = torch.optim.Adam(mlp.parameters(), lr=0.001)
mlp_model = 'best_mlp_model.model'

In [211]:
if PREDICTION == 'node':
    target_train = data.y[data.train_mask].type(torch.long)
    target_val = data.y[data.val_mask].type(torch.long)
    target_test = data.y[data.test_mask].type(torch.long)

## Training the Supervised Learning Network

In [212]:
epochs = 50
validation_loss = 10000.0
for num_epoch in range(epochs):
    mlp_optimizer.zero_grad()
    target = target_train.to(device)
    numbers = list(np.random.randint(500, size=NUM_SAMPLES))
    hidden_samples_train = []
    for number in numbers :
        svd = TruncatedSVD(n_components=256, n_iter=NUM_ITERS, random_state=number)
        u_train = svd.fit_transform(adj_train_corrupted)
        hidden_samples_train.append(torch.Tensor(u_train).to(device))
    for i in range(NUM_SAMPLES):
        hidden_samples_train[i] = torch.cat((hidden_samples_train[i].to(device), data.x[data.train_mask].to(device)),1)
    input_ = torch.stack(hidden_samples_train)
    input_ = input_.detach()
    loss = mlp.compute_loss(input_, target)
    print("Training Loss: ", loss.item())
    with torch.no_grad():
        #Do Validation and check if validation loss has gone down
        numbers = list(np.random.randint(500, size=NUM_SAMPLES))
        hidden_samples_validation = []
        for number in numbers :
            svd = TruncatedSVD(n_components=256, n_iter=NUM_ITERS, random_state=number)
            u_validation = svd.fit_transform(adj_val_corrupted)
            hidden_samples_validation.append(torch.Tensor(u_validation).to(device))
        for i in range(NUM_SAMPLES):
            hidden_samples_validation[i] = torch.cat((hidden_samples_validation[i].to(device), data.x[data.val_mask].to(device)),1)
        input_val = torch.stack(hidden_samples_validation)
        input_val = input_val.detach()
        compute_val_loss = mlp.compute_loss(input_val, target_val.to(device))
        if compute_val_loss < validation_loss:
            validation_loss = compute_val_loss
            print("Validation Loss: ", validation_loss)
            #Save Model
            torch.save(mlp.state_dict(), mlp_model)
    loss.backward()
    mlp_optimizer.step()

Training Loss:  1.9477437734603882
Validation Loss:  tensor(1.9454)
Training Loss:  1.9298110008239746
Validation Loss:  tensor(1.9284)
Training Loss:  1.911606788635254
Validation Loss:  tensor(1.9109)
Training Loss:  1.8903361558914185
Validation Loss:  tensor(1.8912)
Training Loss:  1.8637713193893433
Validation Loss:  tensor(1.8668)
Training Loss:  1.8295457363128662
Validation Loss:  tensor(1.8368)
Training Loss:  1.7886872291564941
Validation Loss:  tensor(1.8015)
Training Loss:  1.7428981065750122
Validation Loss:  tensor(1.7636)
Training Loss:  1.6955639123916626
Validation Loss:  tensor(1.7295)
Training Loss:  1.6480743885040283
Validation Loss:  tensor(1.6978)
Training Loss:  1.5924437046051025
Validation Loss:  tensor(1.6652)
Training Loss:  1.5249956846237183
Validation Loss:  tensor(1.6251)
Training Loss:  1.4495271444320679
Validation Loss:  tensor(1.5773)
Training Loss:  1.3704893589019775
Validation Loss:  tensor(1.5270)
Training Loss:  1.2890057563781738
Validation Los

## Load the best model

In [213]:
mlp = StructMLP(node_set_size).to(device)
mlp.load_state_dict(torch.load(mlp_model))

<All keys matched successfully>

## Forward pass on the test graphs

In [232]:
numbers = list(np.random.randint(500, size=NUM_SAMPLES))
hidden_samples_test = []
for number in numbers :
    svd = TruncatedSVD(n_components=256, n_iter=NUM_ITERS, random_state=number)
    u_test = svd.fit_transform(adj_test_corrupted)
    hidden_samples_test.append(torch.Tensor(u_test).to(device))
for i in range(NUM_SAMPLES):
    hidden_samples_test[i] = torch.cat((hidden_samples_test[i].to(device), data.x[data.test_mask].to(device)),1)
t_test = target_test.to("cpu").numpy()
input_test = torch.stack(hidden_samples_test)
input_test = input_test.detach()

with torch.no_grad():
    test_pred = mlp.forward(input_test)
    pred = F.log_softmax(test_pred)

pred = pred.detach().to("cpu").numpy()
pred = np.argmax(pred, axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


## Test results

In [233]:
print("Test Micro F1 Score: ", f1_score(t_test, pred, average='micro'))
print("Test Weighted F1 Score: ", f1_score(t_test, pred, average='weighted'))
print("Test Accuracy Score: ", accuracy_score(target_test, pred))

Test Micro F1 Score:  0.652
Test Weighted F1 Score:  0.599856541325118
Test Accuracy Score:  0.652
